#### Импорт библиотек, настройка клиента для парсера

In [ ]:
pip install -r requirements_for_parser.txt

In [ ]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import re
import random
import time

import requests 
from bs4 import BeautifulSoup
import urllib.request

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# установите chromedriver для вашей версии Chrome и укажите путь к приложению chromedriver.exe
service = Service(executable_path=r"YOUR_PATH_TO_chromedriver.exe")

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-automation']) 
options.add_experimental_option('useAutomationExtension', False) 
options.add_argument("--disable-blink-features") 
options.add_argument("--disable-blink-features=AutomationControlled") 
options.add_argument('--disable-dev-shm-usage')

#### Парсер для сайта "Автограф"

In [ ]:
class pars_autograph:

    def __init__(self, login=True):

        self.browser = webdriver.Chrome(service=service, options=options)
        self.browser.implicitly_wait(4)

        self.browser.get('http://literature-archive.ru/ru')

        if login==True:

            # Эти x_path могут со временем меняться. 
            # в таком случае необходимо вручную изменить их или выбрать другой метод поиска этих элементов страницы
            user_xp = "/html/body/div[2]/div/div[5]/div/div/div/div[2]/div/section[1]/div/div[2]/form/div/div[1]/input"
            pass_xp = "/html/body/div[2]/div/div[5]/div/div/div/div[2]/div/section[1]/div/div[2]/form/div/div[2]/input"
            log_xp = "/html/body/div[2]/div/div[5]/div/div/div/div[2]/div/section[1]/div/div[2]/form/div/div[4]/input"

            # Необходимо зарегистрироваться и вставить сюда свои данные для входа 
            # это позволит получить более высокое качество изображений
            username = 'YOUR_USERNAME'
            password = 'YOUR_PASSWORD'

            self.browser.find_element(By.XPATH, user_xp).send_keys(username)
            time.sleep(random.uniform(0.2, 0.7))    # можно регулировать
            self.browser.find_element(By.XPATH, pass_xp).send_keys(password)
            time.sleep(random.uniform(0.2, 0.7))    # можно регулировать
            self.browser.find_element(By.XPATH, log_xp).click()
            time.sleep(random.uniform(0.2, 0.7))    # можно регулировать

        soup = BeautifulSoup(self.browser.page_source)

        authors_corpus = soup.find("div", {"class":"main_faces"}).find_all("a", href=True)

        self.authors = {}

        for author in authors_corpus:
            self.authors[author.get('title')] = author.get('href')


    def handle_titles(self, title: str, title_counter: dict):
        if title in title_counter:
            title_counter[title] += 1
            return f"{title}_{title_counter[title]}"
        
        else:
            title_counter[title] = 1
            return title
    

    def getUniqueItems(self, d: dict):
        result = {}
        for key,value in d.items():
            if value not in result.values():
                result[key] = value
        return result


    def pars_author(self, author_name, parent_directory="./data/"):
        
        # Path 
        path = os.path.join(parent_directory, author_name) 
        
        # Create the directory 
        os.mkdir(path) 

        if author_name=='Блок Александр Александрович':

            url_author = BeautifulSoup(requests.get(self.authors[author_name] + '/en/digital-archive/manuscripts').text)
            
        else:

            url_author = BeautifulSoup(requests.get(self.authors[author_name]).text)

        author_archive = url_author.find("div", {"class":"region region-sidebar-first sidebar"}).find_all("a")

        for archive in tqdm(author_archive):
            
            archive_name = re.sub('"', '', archive.text).strip()

            url_archive = BeautifulSoup(requests.get(self.authors[author_name] + archive.get("href")).text)
            
            if url_archive.find("div", {"id":"content"}).find("div", {"class":"view-content"}):
                # Create the directory 
                os.mkdir(os.path.join(parent_directory, author_name, archive_name)) 

                url_path = url_archive.find("div", {"id":"content"}).find("div", {"class":"view-content"})\
                                    .find_all("a", {"class":"language-link"})
                
                material_urls = [(self.authors[author_name] + m_url.get("href")) for m_url in url_path]

                title_path = url_archive.find("div", {"id":"content"}).find("div", {"class":"view-content"})\
                                        .find_all("a", {"rel":"tag"})
                    
                title_counter = {}

                material_titles = [self.handle_titles(m_title.get("title").strip(), title_counter) for m_title in title_path]

                materials_raw = dict(zip(material_titles, material_urls))

                materials = self.getUniqueItems(materials_raw)

                title_counter.clear()

                for title, material in materials.items():

                    title_formated = re.sub(r'[\\/*?:"<>|]', '', title)[:85].strip()

                    unique_title = self.handle_titles(title_formated, title_counter)

                    os.mkdir(os.path.join(parent_directory, author_name, archive_name, unique_title))

                    self.browser.get(materials[title])
                    time.sleep(random.uniform(0.9, 4.1))   # можно регулировать

                    global content

                    content = BeautifulSoup(self.browser.page_source)

                    meta_titles = [mark.text for mark in content.find("div", {"id":"block-system-main"}).find_all("h2")]

                    meta_titles.append("Описание: ")

                    meta_inf = [inf.text for inf in content.find("div", {"id":"block-system-main"}).find_all("div", {"class":"field-item even"})[1:]]

                    self.meta_data = pd.DataFrame([dict(zip(meta_titles, meta_inf))]).T

                    with pd.ExcelWriter(f"{path}/{archive_name}/{unique_title}/meta_data.xlsx") as writer:
                        self.meta_data.to_excel(writer)

                    self.scans = content.find("div", {"id":"block-system-main"}).find_all("a")

                    counter = 1

                    for scan in self.scans:

                        try:
                            image_url = scan.get("href")
                        except:
                            continue

                        if len(image_url) > 50:

                            try:

                                urllib.request.urlretrieve(image_url, f"{path}/{archive_name}/{unique_title}/{counter}.png")
                                counter += 1

                            except:
                                continue
                        
                        else:
                            continue
            else:
                continue

In [ ]:
# Инициализация класса
autoghraph = pars_autograph(login=True)

In [ ]:
# Посмотреть авторов
autoghraph.authors

In [ ]:
target_athors = ['Пастернак Борис Леонидович', 'Булгаков Михаил Афанасьевич', 'Блок Александр Александрович']  # пример

for ath in target_athors:
    # Можно задать другой parent_directory, но рекомендуется хранить данные в каталоге с именем 'Authors_Manusripts'
    autoghraph.pars_author(author_name=ath, parent_directory="./data/Authors_Manusripts/")
    time.sleep(random.uniform(55, 90))  # можно регулировать